In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# load the data
df = pd.read_csv('train.dat',delimiter=' ')
df = df.to_numpy()

In [6]:
df.shape

(39999, 33)

In [3]:
data_trn, data_tst = train_test_split( df, train_size = 30000 )

In [4]:
# data preprocessing include converintg C to X
def preprocess_data( data ):
    X_raw = data[:,:-1]
    y_raw = data[:,-1]
    
    X = np.cumprod( np.flip( 2 * X_raw - 1 , axis = 1 ), axis = 1 )
    y = np.where( y_raw > 0, 1, -1 )
    
    return ( X, y )

In [5]:
( X, y ) = preprocess_data( data_trn )
( X_t, y_t ) = preprocess_data( data_tst )
print(X.shape)
print(X_t.shape)

(30000, 32)
(9999, 32)


In [7]:
def my_map(X):
    # Assume input is n x p matrix 
    # data format is numpy array
    
    # Get the indices for the upper triangle, avoiding the diagonal
    i, j = np.triu_indices(len(X[1, ]), k=1)
    
    # Calculate the pairwise products
    pairwise_products = X[:, i] * X[:, j]
    
    # Concatenate the original X with the pairwise products
    feature_vec = np.append(X, pairwise_products, axis = 1)
    
    return(feature_vec)

In [9]:
Xtrans = my_map(X)
print(Xtrans.shape)

(30000, 528)


# Model fitting

 Linear SVC

In [10]:
import sklearn

In [9]:
from sklearn.svm import LinearSVC
import time
#model1 = LinearSVC(dual=False, max_iter=10000)  # default loss is squared_hinge


model3 = LinearSVC(dual=True ,max_iter=10000, loss='hinge', penalty='l2')
# Fit the model to your data
#model1.fit(Xtrans, y)
start_time2 = time.time()
model2.fit(Xtrans, y)
end_time2 = time.time()
elapsed_time2 = end_time2 - start_time2
print("Sq loss model train time",elapsed_time2)

start_time3 = time.time()
model3.fit(Xtrans, y)
end_time3 = time.time()
elapsed_time3 = end_time3- start_time3
print("hinge loss model train time",elapsed_time3)


Sq loss model train time 1.3001782894134521
hinge loss model train time 10.810450315475464


C:\Users\psand\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [75]:
#from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
import time
#model = LinearSVC(dual=False, max_iter=20000) # for implementing Linear SVC dual must be false for t_train less
model = LogisticRegression(dual=False,max_iter=10000,C=100) # less time
start_time = time.time()
model.fit(Xtrans, y)
end_time = time.time()
print("Sq loss model train time",end_time - start_time)

Sq loss model train time 0.7853827476501465


In [34]:
# checking Accuracy
X_ttrans = my_map(X_t)

y_pred1 = model1.predict(X_ttrans)
print("default model accuracy",np.average( y_t == y_pred1 ))

y_pred2 = model2.predict(X_ttrans)
print("squared_hinge loss model accuracy",np.average( y_t == y_pred2 ))

y_pred3 = model3.predict(X_ttrans)
print("hinge loss model accuracy",np.average( y_t == y_pred3 ))


default model accuracy 0.9905990599059906
squared_hinge loss model accuracy 0.9905990599059906
hinge loss model accuracy 0.987998799879988


In [32]:
help(LinearSVC)

Help on class LinearSVC in module sklearn.svm._classes:

class LinearSVC(sklearn.linear_model._base.LinearClassifierMixin, sklearn.linear_model._base.SparseCoefMixin, sklearn.base.BaseEstimator)
 |  LinearSVC(penalty='l2', loss='squared_hinge', *, dual=True, tol=0.0001, C=1.0, multi_class='ovr', fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=1000)
 |  
 |  Linear Support Vector Classification.
 |  
 |  Similar to SVC with parameter kernel='linear', but implemented in terms of
 |  liblinear rather than libsvm, so it has more flexibility in the choice of
 |  penalties and loss functions and should scale better to large numbers of
 |  samples.
 |  
 |  This class supports both dense and sparse input and the multiclass support
 |  is handled according to a one-vs-the-rest scheme.
 |  
 |  Read more in the :ref:`User Guide <svm_classification>`.
 |  
 |  Parameters
 |  ----------
 |  penalty : {'l1', 'l2'}, default='l2'
 |      Specifies t

Logistic regression

In [37]:
from sklearn.linear_model import LogisticRegression
help(LogisticRegression)

Help on class LogisticRegression in module sklearn.linear_model._logistic:

class LogisticRegression(sklearn.linear_model._base.LinearClassifierMixin, sklearn.linear_model._base.SparseCoefMixin, sklearn.base.BaseEstimator)
 |  LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
 |  
 |  Logistic Regression (aka logit, MaxEnt) classifier.
 |  
 |  In the multiclass case, the training algorithm uses the one-vs-rest (OvR)
 |  scheme if the 'multi_class' option is set to 'ovr', and uses the
 |  cross-entropy loss if the 'multi_class' option is set to 'multinomial'.
 |  (Currently the 'multinomial' option is supported only by the 'lbfgs',
 |  'sag', 'saga' and 'newton-cg' solvers.)
 |  
 |  This class implements regularized logistic regression using the
 |  'liblinear' library, 'newton-cg', 's

In [60]:
#model = LogisticRegression(dual=False,max_iter=10000)
#model = LogisticRegression(dual=False,max_iter=10000,penalty="l2")
#model = LogisticRegression(dual=False,max_iter=10000,penalty="none")
#model = LogisticRegression(dual=False,max_iter=10000,penalty="l2",solver='newton-cholesky')
#model = LogisticRegression(dual=False,max_iter=10000,penalty=None,solver='newton-cholesky')
#model = LogisticRegression(dual=False,max_iter=10000,penalty="l1",solver='liblinear') # taking too much time
#model = LogisticRegression(dual=False,max_iter=10000,penalty="l1",solver='saga') # too much time
model.fit(Xtrans,y)

LogisticRegression(max_iter=10000, penalty='l1', solver='saga')

In [61]:
y_pred = model.predict(X_ttrans)
print("model accuracy",np.average( y_t == y_pred )) # accuracy

model accuracy 0.9902990299029903
